In [44]:
import os

import numpy as np
import pandas as pd

In [22]:
def getFiles(filepath):
    files = []
    for file in os.listdir(data_path):
        if not file.startswith('.') and not file.endswith('.zip'):
            files.append(os.path.join(data_path, file))
    return files

In [98]:
def filterFiles(files, date):
    
    files_aqua = []
    
    for file in files:
        # read in
        df = pd.read_csv(file).reset_index()
        # set correct column names, drop row 0
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        # make date object
        df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
        filtered = df[df['date'] >= date]
        
        if filtered.shape[0] != 0:
            files_aqua.append(file)
        
    return files_aqua

In [99]:
data_path = '/nas/cee-water/cjgleason/data/Riggs_gauges/'

In [100]:
files = getFiles(data_path)

In [ ]:
files_aqua = filterFiles(files=files, date='2002-07-04')

In [55]:
test.columns = test.iloc[0]

In [57]:
test = test.iloc[1:]

In [64]:
test

,Year,Month,Day,Discharge (m^3/s),date
1,1966,10,1,0,1966-10-01
2,1966,10,2,0,1966-10-02
3,1966,10,3,0,1966-10-03
4,1966,10,4,0,1966-10-04
5,1966,10,5,0,1966-10-05
...,...,...,...,...,...
13972,2021,9,3,0,2021-09-03
13973,2021,9,4,0,2021-09-04
13974,2021,9,5,0,2021-09-05
13975,2021,9,6,0,2021-09-06


In [67]:
test['date'] = pd.to_datetime(test[['Year', 'Month', 'Day']])

/tmp/ipykernel_3768475/3608695399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'] = pd.to_datetime(test[['Year', 'Month', 'Day']])


In [69]:
test[test['date'] >= '2002-07-04']

,Year,Month,Day,Discharge (m^3/s),date
8036,2005,6,3,0,2005-06-03
8037,2005,6,4,0,2005-06-04
8038,2005,6,5,0,2005-06-05
8039,2005,6,6,0,2005-06-06
8040,2005,6,7,0,2005-06-07
...,...,...,...,...,...
13972,2021,9,3,0,2021-09-03
13973,2021,9,4,0,2021-09-04
13974,2021,9,5,0,2021-09-05
13975,2021,9,6,0,2021-09-06


In [53]:
test.iloc[1:,0:3]

,index,Latitude: -22.910194,Longitude: 139.915972
1,1966,10,1
2,1966,10,2
3,1966,10,3
4,1966,10,4
5,1966,10,5
...,...,...,...
13972,2021,9,3
13973,2021,9,4
13974,2021,9,5
13975,2021,9,6


In [49]:
np.array(test['index'][1:].astype(int))

array([1966, 1966, 1966, ..., 2021, 2021, 2021])